In [14]:
# https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/pdf.html
import pathlib
import glob
import shutil
import os, sys
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
load_dotenv('./.env')

# 이렇게 하면 API key 를 노출시키지 않고 사용할 수 있다.
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [7]:
pdf_files = []
for pdf_file_namedir in glob.glob('./../ref_docs/*.pdf', recursive=True):
    pdf_files.append(pdf_file_namedir)
pdf_files

['./../ref_docs/2022년 가계동향조사 지침서.pdf',
 './../ref_docs/이용자용 통계정보보고서_사망원인통계_2020.pdf',
 './../ref_docs/이용자용 통계정보보고서_가계동향조사_2021.pdf',
 './../ref_docs/0523 생활시간 조사표대표가구.pdf',
 './../ref_docs/2022년 가계동향조사 조사표.pdf',
 './../ref_docs/이용자용 통계정보보고서_생활시간조사.pdf',
 './../ref_docs/2019년 생활시간조사 지침서.pdf',
 './../ref_docs/생활시간조사_조사분석기법탐구-강유경.pdf']

In [39]:
# 로더, 텍스트 분할기 및 임베딩 초기화
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
embeddings = OpenAIEmbeddings()

texts = []

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents = loader.load()
    texts.extend(text_splitter.split_documents(documents))

# pdf 로 텍스트를 가져오고, 스플릿하고, 임베딩을 생성합니다.
docsearch = Chroma.from_documents(texts, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [58]:
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

from langchain.prompts import PromptTemplate
prompt_template = """
다음 주어지는 문맥을 사용하여 주어지는 질문에 답하세요.
만약 답을 모르면 모른다고 하세요.
주어진 답이 표 형태면 Markdown 표 형태로 작성하세요.

{context}

질문: {question}
답변:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0.7), 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=True, 
                                 chain_type_kwargs=chain_type_kwargs)

In [59]:
query = "생활시간조사에서 쓰는 행동분류표 예시"
result = qa({"query": query})
print(result["result"])
print('\n')
result["source_documents"]

| 분류 | 항목 |
|---|---|
| 주로 한 행동 | 스마트폰, 태블릿, PC, 노트북 |
| 동시에 한 행동 | 주로 한 행동 외 다른 행동 |
| 장소 또는 이동수단 | 자기 집, 직장, 학교, 남의 집, 기타(실내), 기타(실외), 도보, 버스, 지하철/철도, 택시, 승용차, 자전거, 기타 교통 |
| 함께한 사람 | 혼자, 배우자, 만10세 미만(손)자녀, 만10세 이상(손)자녀, 부모(배우자 쪽 포함), 형제자매/기타가족, 기타 아는 사람 |
| ICT기기 | 스마트폰, 태블릿, PC, 노트북 |




[Document(page_content='2019년 생활시간조사 표(대표 가구원)12\n※ 통계청 기입  1. 주로 한 행동 2. 동시에 한 행동     3.   장소 또는 \n이동수단4. 함께한 사람ICT기기 ICT기기\n주\n행동동시\n행동10분 동안 주로 한 행동을 씁니다.1.   스마트폰,  \n태블릿\n2.   PC,  \n노트북주로 한 행동외에\n다른 행동을 동시에 한 경우 씁니다.1.   스마트폰,  \n태블릿\n2.   PC,  \n노트북1. 자기 집\n2. 직장\n3. 학교\n4. 남의 집\n5. 기타(실내)\n6. 기타(실외)\n7. 도보 8. 버스\n 9.   지하철 ·  \n철도\n10. 택시\n11. 승용차\n12. 자전거\n13. 기타 교통1. 혼자\n2. 배우자\n3. 만10세 미만 (손)자녀\n4. 만10세 이상 (손)자녀\n5. 부모(배우자 쪽 포함)\n6. 형제자매, 기타가족\n9. 기타 아는 사람\n109\n110\n111\n112\n113\n114\n115\n116\n117\n118\n119\n120\n121\n122\n123\n124\n125\n126\n127\n128\n129\n130\n131\n132\n133\n134\n135\n136\n137\n138\n139\n140\n141\n142\n143\n144저녁\n6시\n10\n20\n30\n40\n50\n7시\n10\n20\n30\n40\n50\n8시\n10\n20\n30\n40\n50\n9시\n10\n20\n30\n40\n50\n10시 \n10\n20\n30\n40\n50\n11시 \n10\n20\n30\n40\n50\n12시 첫째 날 저녁 6시 ~ 밤 12시', metadata={'source': './../ref_docs/0523 생활시간 조사표대표가구.pdf', 'page': 11}),
 Document(page_content='2019년 생활시간조사 표(대표 가구원)14\n※ 통계청 기입  1. 주로 한 행동 2. 동시에 한 행동   

In [60]:
query = "범주화된 행동분류"
result = qa({"query": query})
print(result["result"])
print('\n')
result["source_documents"]

| 대분류 | 범주 |
| --- | --- |
| 필수시간 | 수면, 식사 및 간식, 기타 개인유지 |
| 의무시간 | 일(수입노동), 가사노동, 학습, 이동 |
| 여가시간 | 교제, 미디어 이용, 종교/문화/스포




[Document(page_content='80｜http://www.narastat.kr/time2019\n행동분류가 애매한 경우\n  -  부록의 행동분류표 또는 행동분류 사례집을 보고 판단하거나 , ‘지도공무원 ’ 또는 \n‘본부실사지도 공무원 ’에게 문의\n  -  생활시간조사 시스템의 게시판 > 질의응답 코너에 질의\n※ 아래의 예시는 지하철 기관사와 놀이공원에서의 행동을 분류한 것임\n< 지하철 기관사의 행동 >\n․ 출근하는 행동 ⇒ 출근 (921)\n․ 조회 및 업무를 배정받는 행동 ⇒ 법인 , 정부기관 등의 일 (210)\n․ 지하철을 운행 (노량진역 ↔ 신길역 ) ⇒ 법인 , 정부기관 등의 일 (210)\n․ 퇴근을 위해 도시지하철공사로 이동 ⇒ 기타 일 관련 이동 (923)\n< 놀이공원 내의 행동 >\n․ 주차장에 도착하여 셔틀 버스를 타기 위해서 이동하거나 줄을 서서 기다리는 행동 \n  ⇒ 교제 ·여가활동 관련 이동 (971)\n․ 놀이기구를 타기  ⇒ 기타 스포츠 ·레포츠 (839)\n․ 놀이기구를 타기 위하여 이동하거나 기다리는 행동  ⇒ 기타 스포츠 ·레포츠 (839)\n․ 먹을 것을 구입하기 위하여 줄을 서는 행동 ⇒ 가정관리 관련 이동 (940)\n․ 잃어버린 아이 찾기 ⇒ 기타 돌보기 (519)\n․ 먹을 것을 먹거나 화장실에 가는 행동 ⇒ 간식 ․음료(122), 개인위생 (141)\n․ 물놀이장에서 샤워하는 행동 ⇒ 개인위생 (141)\n․ 물놀이장에서 옷을 갈아입는 행동 ⇒ 외모관리 (142)', metadata={'source': './../ref_docs/2019년 생활시간조사 지침서.pdf', 'page': 83}),
 Document(page_content='- 66 -○행동분류 범주화\n -국가별로 행동분류 체계를 재분류하여 범주화한 것으로 각 국이 모두 상이함\n<행동분류 범주화 >\n한국(3) 일본(3)*호주(4) 미국(12)\n필수시간\n 수면\n 식사 및 간식\n 기타 개인유지\n

In [61]:
query = "행동분류코드"
result = qa({"query": query})
print(result["result"])
print('\n')
result["source_documents"]

행동분류코드는 세 자리로 구성되며, 첫 자리는 대분류, 두 번째 자리는 중분류, 세 번째 자리는 소분류를 나타냅니다. 행동분류는 행동의 목적에 따라 분류하며, 대분류를 먼저 찾은 후 대분류 내에서 중분류를 결정하고 마지막 단계에서 소분류를 찾아 나가는 방법으로 분류합니다. 이동 관련 행동분류 코드의 첫 자리(대분류)는 항상 ‘9’이며, 두 번째 자리(중분류)는 이동하고자 하는 목적에 따라 행동분류표에서 찾아 써 넣습니다. 대부분의 경우, 이동의 중분류 자리는 다음 행동의 대분류 자리와 동일합니다.




[Document(page_content='78｜http://www.narastat.kr/time2019\n○ 행동분류 코드 기입 예시\n-동일한 행동을 하더라도 행동의 목적 , 응답자의 연령이나 가족과 함께 사는지 여부 등에 \n따라 분류가 달라질 수 있음\n행동 세부내용 코드 행동 세부내용 코드\nTV\n시청․교육방송시청\n․상품 홈쇼핑 시청\n․실시간 드라마시청\n․드라마 , 예능 등 다시보기\n․종교방송 시청322\n472\n824\n825731전화통화․어머니 문안전화\n․식료품 주문\n․짜장면 주문\n․컴퓨터 A/S 신청\n․숙박 예약712\n472\n474\n443474\n스마트폰 \n사용\n(전화 \n통화 \n제외)․모바일 게임\n․영화표 예매\n․스포츠경기 생중계 보기\n․친구와 카톡\n․친구 인스타그램 구경하기\n․(회사원 ) 거래처와 업무상 문자\n․인터넷 뉴스 보기\n․인터넷 서핑843\n474824\n713\n714210\n822\n828산책․남편과 공원 산책하기\n․8살 자녀와 산책하기\n․반려견과 산책하기831\n515\n461\n돌봄․8세인 딸 간호하기\n․1 5세인 딸 간호하기\n․요양병원에 입원한 엄마 약 \n챙겨드리기\n․감기로 편찮으신 엄마 약 \n챙겨드리기512\n522\n532\n541\n❍이동 관련 기입 예시 (1)\n-이동은 이동목적에 따라 코드가 변경될 수 있으므로 이동의 목적을 함께 기입하도록 지도하며  \n이동수단을 반드시 기입하도록 함\n-마트에서 물건을 구입하는 것은 3분이 소요되었으나 마트까지의 이동은 10분이 소요된 \n경우에 구매한 행동은 나타나지 않고 이동만 나타날 수 있음\n행동 행동분류 코드 이동수단 코드\n장보기 위하여 집에서 마트로 이동 940. 가정관리 관련 이동 시내버스 8\n공부하기 위하여 마트에서 도서관으로 이동 930. 학습 관련 이동 지하철 9\n부모님을 돌보기 위하여 부모님 댁으로 이동 954. 독립적인 성인 돌보기 관련 이동 자전거 12\n4세 자녀를 데리러 가기 위하여 어린이집으로